In [1]:
import pandas as pd
import numpy as np
from scipy import stats
from joblib import dump, load

# Data visualization
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style="whitegrid")

# Sklearn imports
from sklearn.multiclass import OneVsRestClassifier
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
# old classification metrics
from sklearn.metrics import ( classification_report, confusion_matrix, 
                             precision_score, accuracy_score, 
                             precision_recall_fscore_support )
# new classification metrics
from sklearn.metrics import make_scorer, roc_curve, roc_auc_score

In [2]:
def get_metrics(confusion_matrix_file_name):
    '''
        read a confusion matrix from a file, and calculate some metrics
        input: a pd.DataFrame representing a confusion matrix
        output: a pd.DataFrame containing the metrics
    '''
    confusion_matrix = pd.read_csv(confusion_matrix_file_name)
    
    if 'Unnamed: 0' in confusion_matrix.columns:
        confusion_matrix.drop(columns=['Unnamed: 0'], inplace=True)
    
    confusion_matrix = confusion_matrix.values
    metrics = {}
    
    FP = metrics['False Positive'] = confusion_matrix.sum(axis=0) - np.diag(confusion_matrix)  
    FN = metrics['False Negative'] = confusion_matrix.sum(axis=1) - np.diag(confusion_matrix)
    TP = metrics['True Positive'] = np.diag(confusion_matrix)
    TN = metrics['True Negative'] = confusion_matrix.sum() - (FP + FN + TP)

    # Sensitivity, hit rate, recall, or true positive rate
    metrics['True Positive Rate'] = TP/(TP+FN)
    # Specificity or true negative rate
    metrics['True Negative Rate'] = TN/(TN+FP) 
    # Precision or positive predictive value
    metrics['Precision'] = TP/(TP+FP)
    # Fall out or false positive rate ##########
    metrics['False Positive Rate'] = FP/(FP+TN)
    # False negative rate ############
    metrics['False Negative Rate'] = FN/(TP+FN)
    # Overall accuracy
    metrics['Overall Accuracy'] = (TP+TN)/(TP+FP+FN+TN)
    
    #for metric, values in metrics.items():
    #print(f'{metric} has {len(values)} lines\n')
    
    metrics_df = pd.DataFrame(metrics)

    return metrics_df

In [ ]:
metrics_step_1 = get_metrics('confusion_matrix_1.csv')
metrics_step_2 = get_metrics('confusion_matrix_2.csv')
metrics_step_3 = get_metrics('confusion_matrix_3.csv')
metrics_step_4 = get_metrics('confusion_matrix_4.csv')

In [62]:
# plot metrics here

In [ ]:
def get_roc_auc_score(y_test, probabilities):
    '''
        Calculate 4 different averages for the roc auc score
    '''
    # https://scikit-learn.org/stable/auto_examples/model_selection/plot_roc.html#area-under-roc-for-the-multiclass-problem
    
    roc_auc = {}
    roc_auc['OVR Macro Average'] = roc_auc_score(y_test, probabilities, 
                                      multi_class="ovr", average="macro")
    roc_auc['OVR Weighted Average'] = roc_auc_score(y_test, probabilities, 
                                         multi_class="ovr", average="weighted")
    roc_auc['OVR Micro Average'] = roc_auc_score(y_test, probabilities, 
                                      multi_class="ovr", average="micro")
    roc_auc['OVR Samples Average'] = roc_auc_score(y_test, probabilities, 
                                         multi_class="ovr", average="samples")

    return roc_auc


In [1]:
# save roc auc scores
roc_auc_step_1 = get_roc_auc_score(y_test_step_1, probabilities_step_1)
roc_auc_step_2 = get_roc_auc_score(y_test_step_2, probabilities_step_2)
roc_auc_step_3 = get_roc_auc_score(y_test_step_3, probabilities_step_3)
roc_auc_step_4 = get_roc_auc_score(y_test_step_4, probabilities_step_4)

# make a dataframe containing roc_auc_scores from all 4 steps
index = ['Step 1', 'Step 2', 'Step 3', 'DETECT']
roc_auc_scores_df = pd.Dataframe([roc_auc_step_1, roc_auc_step_2, roc_auc_step_3, roc_auc_step_4], index=index)

NameError: name 'get_roc_auc_score' is not defined

In [ ]:
#plot roc_auc_score here